In [11]:
import os

In [1]:
%pwd

'c:\\Users\\Mehdi\\Desktop\\Text-Summarizer\\research'

In [13]:

%pwd

'c:\\Users\\Mehdi'

In [25]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [26]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [27]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config
        

In [28]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [29]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded ! With following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        """ 
        zip_file_path: str
        Extract the contents of the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [14]:
os.chdir("../")

In [30]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-15 20:17:15,291 : INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-15 20:17:15,292 : INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-15 20:17:15,293 : INFO: common: Directory: artifacts already exists]
[2024-10-15 20:17:15,294 : INFO: common: Directory: artifacts/data_ingestion already exists]
[2024-10-15 20:17:17,369 : INFO: 2215193081: artifacts/data_ingestion/data.zip downloaded ! With following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 53A6:3978D4:1757223:185AC2A:670EB1AA
Accept-Ranges: bytes
Date: Tue, 15 Oct 2024 18:17:16 GMT
Via: 1.1 varnish
X-Served-By: cache-par-l